# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `4`
* Students: `Julien Berger, Jérémy Jayet, Hanah Samet, Mathieu Shiva`
* Dataset: `IMDb Films and Crew `

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

In [2]:
# nodes = cast/crew 
# edges = co-appearance in TV/movies
# features = rating of movies
# labels = movie genres
movies = pd.read_csv('../data/tmdb_5000_movies.csv')
movies['release_date'] = pd.to_datetime(movies['release_date']).apply(lambda x: x.date())
json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
for column in json_columns:
    movies[column] = movies[column].apply(json.loads)


credits = pd.read_csv('../data/tmdb_5000_credits.csv')
json_columns = ['cast', 'crew']
for column in json_columns:
       credits[column] = credits[column].apply(json.loads)

def safe_access(container, index_values):
    # return a missing value rather than an error upon indexing/key failure
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan
    
#credits['gender_of_lead'] = credits.cast.apply(lambda x: safe_access(x, [0, 'gender']))
#credits['lead'] = credits.cast.apply(lambda x: safe_access(x, [0, 'name']))
#df = pd.merge(movies, credits, left_on='id', right_on='movie_id')
#df[['original_title', 'revenue', 'lead', 'gender_of_lead']].sort_values(by=['revenue'], ascending=False)[:10]

credits.apply(lambda row: [x.update({'movie_id': row['movie_id']}) for x in row['cast']], axis=1);
credits.apply(lambda row: [x.update({'movie_id': row['movie_id']}) for x in row['crew']], axis=1);
credits.apply(lambda row: [person.update({'order': order}) for order, person in enumerate(row['crew'])], axis=1);

cast = []
credits.cast.apply(lambda x: cast.extend(x))
cast = pd.DataFrame(cast)
cast['type'] = 'cast'

crew = []
credits.crew.apply(lambda x: crew.extend(x))
crew = pd.DataFrame(crew)
crew['type'] = 'crew'

people = pd.concat([cast, crew],  ignore_index=True, sort=False)
people = people.drop(columns=['gender','department', 'credit_id','cast_id', 'job','order','character','type'])
people = people.sort_values(by='id')

#remove the rows with similar id and movie
people=people.drop_duplicates(subset=['id', 'movie_id'])

#get the maximum number of movies someone worked on, this will be the number of features we take
maximum_movies=people['id'].value_counts().tolist()
maximum_movies=maximum_movies[0]
table_nb_movies=people['id'].value_counts()
unique_values=people['id'].unique()

        
        
#get the total number of individual "couples" movie+person
number_entries=len(people.index)
movies['movie_id']=movies['id']
movies=movies.drop(columns=['vote_count','budget','genres','homepage','keywords','original_language','overview','popularity','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','status','tagline','original_title'])
movies = movies.set_index('movie_id')
  
#merge the movies and the people so that you have the rating    
people = people.merge(movies, on='movie_id')
people['id']=people['id_x']
people = people.drop(columns=['id_x','id_y'])


#this simple_list will contain all the different actor names
simple_list=people.loc[:, ['id','movie_id','name']]
simple_list=simple_list.sort_values(by='id')
simple_list=simple_list.drop_duplicates('id')
simple_list=simple_list.set_index('id') 
simple_list=simple_list.drop(columns=['movie_id'])

#Only 35'000 people did more than 1  movie
#threshold = 5 movies, gives a list of around 9600 people --> good





In [3]:


#This code takes 14 minutes to run and removes all the people that worked on less than 5 movies
threshold_movies=5
for idx in unique_values:
    nb_films=table_nb_movies[idx] 
    if (nb_films)<threshold_movies:
        simple_list=simple_list.drop(index=idx)


In [4]:

#Calculate the number of individual movies and people
unique_id=people['id'].unique()
unique_id.sort()
number_people=len(simple_list)

#this isn't usefull anymore

#The size is 104842 --> number of unique people(before cleaning, after it is 9628)
#unique_movie_id=people['movie_id'].unique()
#unique_movie_id.sort()
#number_movies=unique_movie_id.size
#The size is 4782 --> number of unique movies

#Add a column that will contain the average rating of the actor 
simple_list['Average Rating']=np.nan

#Add a column for each movie (based on the highest number of movie for 1 individual)
for i in range(maximum_movies):
    simple_list['Movie %d' % i]=np.nan



In [5]:
#This code takes 5 minutes to run and adds all the movies ID on the correct row of simple list

# We take a subset of the big list (people) with all the movies 1 person starred in (or worked on)
# We then place them in the corresponding row in the simple_list
# We also calculate the Average rating of the actor and place it in the simple_list
for idx in simple_list.index.values:
    rating_average=0
    subset=people[people['id'] == idx]
    new_index_subset = pd.Series(range(0,len(subset)))
    subset=subset.set_index(new_index_subset) 

    for i in new_index_subset:
        simple_list.loc[idx, 'Movie %d' % i]=subset.loc[i,'movie_id']
        rating_average=rating_average+subset.loc[i,'vote_average']
    rating_average=rating_average/len(subset)
    simple_list.loc[idx, 'Average Rating']=rating_average
    
simple_list

#we save this to a csv for later use    
#simple_list.to_csv('simple_list.csv')    

#This simple list is the features we want to use    
#features = # the pandas dataframe with the features and labels

,name,Average Rating,Movie 0,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,...,Movie 73,Movie 74,Movie 75,Movie 76,Movie 77,Movie 78,Movie 79,Movie 80,Movie 81,Movie 82
id,,,,,,,,,,,,,,,,,,,,,
1,George Lucas,6.955556,13155.0,217.0,85.0,87.0,306.0,1893.0,838.0,10658.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Harrison Ford,6.634375,217.0,85.0,87.0,838.0,89.0,1892.0,1891.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Carrie Fisher,6.218750,1892.0,879.0,1891.0,11.0,2294.0,639.0,525.0,17926.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Anthony Daniels,7.437500,1893.0,1894.0,1895.0,1892.0,333355.0,1891.0,11.0,137106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Andrew Stanton,7.328571,10681.0,10193.0,9806.0,12.0,585.0,150540.0,862.0,62211.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Lee Unkrich,7.357143,10193.0,12.0,585.0,862.0,62211.0,863.0,9487.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Bob Peterson,7.425000,10193.0,9806.0,12.0,585.0,150540.0,62211.0,14160.0,105864.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Albert Brooks,6.614286,12.0,328111.0,1389.0,61337.0,241239.0,35.0,15301.0,37718.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Brad Garrett,6.146154,12.0,2062.0,9487.0,38757.0,10022.0,124459.0,181533.0,308531.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
simple_list.to_csv('simple_list_9628.csv') 

### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in the  [graph from edge list]() demo.

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. In the [graph from features]() demo Benjamin showed you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [ ]:
# Your code here

n_nodes=5
#n_nodes = len(nodes)
#n_nodes=len(simple_list)
adjacency = np.zeros((5, 5), dtype=int)

#for idx, row in edges.iterrows():
#    if np.isnan(row.node_idx_parent):
#        continue
#    i, j = int(row.node_idx), int(row.node_idx_parent)
#    adjacency[i, j] = 1
#    adjacency[j, i] = 1
adjacency[1,2]=1  
adjacency[2,1]=1  
adjacency[0,3]=1  
adjacency[3,0]=1  
adjacency[4,0]=1  
adjacency[0,4]=1  
adjacency[4,2]=1  
adjacency[2,4]=1  


adjacency

#adjacency = # the adjacency matrix
#n_nodes = # the number of nodes in the network

## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [ ]:
plt.spy(adjacency, markersize=1)
plt.title('adjacency matrix')

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [ ]:
#use the unweighted matrix
L=sum(adjacency)
L=sum(L)/2 #the number of links in the network
L_max=pd.Series(range(0,n_nodes))
L_max=sum(L_max)
L_max

The maximum number of links in a network is equal to the sum off all number from 1 to N-1 (1+2+3+...+N-2+N-1)
In our case, since N=..., L_max=...
In our network we L links, which means that our network is really sparse in comparison

### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

It is an undirected graph.

In [ ]:
# Your code here.

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [ ]:
#features.to_csv('features.csv')
#np.save('adjacency.npy', adjacency)

### Question 4

Are the edges of your graph weighted?

Yes, the weights are equal to the number of movies that two people have in common.

### Question 5

What is the degree distibution of your network? 

In [ ]:
degree =  # Your code here. It should be a numpy array.

assert len(degree) == n_nodes

Execute the cell below to see the histogram of the degree distribution.

In [ ]:
weights = np.ones_like(degree) / float(n_nodes)
plt.hist(degree, weights=weights);

What is the average degree?

In [ ]:
# Your code here.

### Question 6

Comment on the degree distribution of your network.

**Your answer here.**

### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [ ]:
def breadth_first_search(adjacency, source, destination):
    nodeList = adjacency.shape
    nodeList = nodeList[0]
    nodeList = np.full(nodeList, np.nan)

    queue = np.nonzero(adjacency[source,:])
    queue = np.array(queue[0])

    k = 1

    if source == destination :
        nodeList[destination] = 0

    while np.isnan(nodeList[destination]):
        for i in np.nditer(queue) :
            #print("node = ", i)
            if np.isnan(nodeList[i]) :
                nodeList[i] = k
                tmp = np.nonzero(adjacency[i,:])
                tmp = np.array(tmp[0])
                #print(queue, "+" ,tmp)
                queue = np.concatenate((queue, tmp))


        k = k+1

    distance = nodeList[destination]
    return distance




def connected_graph(adjacency):
    """Determines whether a graph is connected.

    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.

    Returns
    -------
    bool
        True if the graph is connected, False otherwise.
    """

    nodeList = adjacency.shape
    nodeList = nodeList[0]
    nodeList = np.full(nodeList, np.nan)

    nodeList[0] = 1;

    for i in range(0,nodeList.size):
        for j in range(i+1,nodeList.size):
            #print("i=",i,"j=",j, nodeList[j] ,np.isnan(nodeList[j]))
            if nodeList[i] == 1 and np.isnan(nodeList[j]):
                if np.isnan(breadth_first_search(adjacency,i,j)) == False:
                    nodeList[j] = 1
                else:
                    nodeList[j] = 0

    #print(nodeList)

    if np.sum(nodeList) == nodeList.size:
        connected = True
    else:
        connected = False


    return connected

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [ ]:
# Your code here.

### Question 8

Write a function that extracts the connected components of a graph.

In [ ]:
def find_components(adjacency):
    """Find the connected components of a graph.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    list of numpy arrays
        A list of adjacency matrices, one per connected component.
    """
    
    # Your code here.
    
    return components

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

In [ ]:
# Your code here.

### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [ ]:
def compute_shortest_path_lengths(adjacency, source):
    """Compute the shortest path length between a source node and all nodes.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    
    Returns
    -------
    list of ints
        The length of the shortest path from source to all nodes. Returned list should be of length n_nodes.
    """
    
    # Your code here.
    
    return shortest_path_lengths

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [ ]:
# Your code here.

### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

In [ ]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    
    # Your code here.
    
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

In [ ]:
print(compute_paths(adjacency, 0, 10, 1))
print(compute_paths(adjacency, 0, 10, 2))
print(compute_paths(adjacency, 0, 10, 3))
print(compute_paths(adjacency, 23, 67, 2))
print(compute_paths(adjacency, 15, 93, 4))

### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [ ]:
# Your code here.

### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

In [ ]:
def compute_clustering_coefficient(adjacency, node):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    
    # Your code here.
    
    return clustering_coefficient

### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [ ]:
# Your code here.